In [ ]:
 current_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis"

In [ ]:
import os
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
os.chdir(current_dir)

Mounted at /content/gdrive


In [ ]:
target_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis/Politicians"

In [ ]:
os.chdir(current_dir)

In [ ]:
ls

 logistic_regression.ipynb        politician_links.ipynb       'Sound Files'/
 Meetings/                        Politicians/                 'Thesis Reports'/
'Politicans: Baselines.gslides'   politicians_baselines.ipynb


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
# .load assigns an analysis pipeline to the function nlp
nlp = spacy.load("en_core_web_sm") # This is one of the pipelines for English
spacy.__version__


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-12-08 01:34:29.751677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 01:34:29.751756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: A

'3.7.2'

In [ ]:
text = "Airstrikes targeting a residential neighborhood in and around Damascus, \
Syria’s capital, killed several people and damaged residential buildings. \
Syrian state media and the Syrian Observatory for Human Rights reported this early on Sunday."

doc = nlp(text)
doc

for token in doc:
    print(token.pos_, token.lemma_, token.dep_, token.ent_type_,)

NOUN airstrike nsubj 
VERB target ROOT 
DET a det 
ADJ residential amod 
NOUN neighborhood dobj 
ADP in prep 
CCONJ and cc 
ADP around conj 
PROPN Damascus pobj GPE
PUNCT , punct 
PROPN Syria poss GPE
PART ’s case 
NOUN capital npadvmod 
PUNCT , punct 
VERB kill conj 
ADJ several amod 
NOUN people dobj 
CCONJ and cc 
VERB damage amod 
ADJ residential amod 
NOUN building conj 
PUNCT . punct 
ADJ syrian amod NORP
NOUN state compound 
NOUN medium nsubj 
CCONJ and cc 
DET the det ORG
PROPN Syrian compound ORG
PROPN Observatory conj ORG
ADP for prep ORG
PROPN Human compound ORG
PROPN Rights pobj ORG
VERB report ROOT 
PRON this dobj DATE
ADV early advmod DATE
ADP on prep 
PROPN Sunday pobj DATE
PUNCT . punct 


In [ ]:
def complexity (txt):
    doc = nlp(txt, disable=['lemmatizer', 'ner'])
    sent_len_values = [len(sent) for sent in doc.sents]
    word_len_values = [len(token) for token in doc]
    return sum(word_len_values)/len(word_len_values)


def pos_tag (txt):
    doc = nlp(txt, disable=['lemmatizer', 'ner'])
    return " ".join([token.pos_ for token in doc])

In [ ]:
# Change the current working directory to the 'Politicians' folder
folder_path = target_dir  # Replace with your actual folder path
os.chdir(folder_path)

# List all txt files in the current directory
file_names = [f for f in os.listdir() if f.endswith('.txt')]
file_contents = []

# Read the contents of each txt file
for file_name in file_names:
    with open(file_name, 'r') as file:
        file_contents.append(file.read())

In [ ]:
database = {}


for name, content in zip(file_names, file_contents):
    name_without_extension = name.split('.txt')[0]
    database[name_without_extension] = content

In [ ]:
database.keys()

dict_keys(['jfk', 'reagan', 'fdr', 'clinton', 'gwb', 'eisenhower', 'lbj', 'ford', 'carter', 'hwb', 'truman', 'hoover', 'cruz', 'sanders', 'trump', 'nixon'])

In [ ]:
real_name = {
    'jfk': 'John F. Kennedy',
    'reagan': 'Ronald Reagan',
    'fdr': 'Franklin D. Roosevelt',
    'clinton': 'William Jefferson Clinton',
    'gwb': 'George W. Bush',
    'eisenhower': 'Dwight D. Eisenhower',
    'lbj': 'Lyndon B. Johnson',
    'ford': 'Gerald Ford',
    'carter': 'James Earl Carter',
    'nixon': 'Richard Nixon',
    'hwb': 'George H. Bush',
    'truman': 'Harry S. Truman',
    'hoover': 'Herbert Hoover',
    'cruz': 'Ted Cruz',
    'sanders': 'Bernie Sanders',
    'trump': 'Donald Trump'
}


In [ ]:
import re
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Replace newlines and double whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
texts = {real_name[k]: clean_text(v) for k, v in database.items()}


In [ ]:
for k,v in texts.items():
  print(k, len(v.split()))

John F. Kennedy 809
Ronald Reagan 749
Franklin D. Roosevelt 283
William Jefferson Clinton 685
George W. Bush 399
Dwight D. Eisenhower 576
Lyndon B. Johnson 586
Gerald Ford 507
James Earl Carter 411
George H. Bush 1025
Harry S. Truman 970
Herbert Hoover 1500
Ted Cruz 812
Bernie Sanders 832
Donald Trump 1761
Richard Nixon 1551


In [ ]:
dutton_scores = {
    'Donald Trump': 22.67,
    'Ted Cruz': 20.33,
    'Bernie Sanders': 18.0
}


In [ ]:
lilienfeld_scores = {
    "Theodore Roosevelt": 1.462,
    "John F. Kennedy": 1.408,
    "Franklin D. Roosevelt": 1.079,
    "Ronald Reagan": 0.912,
    "Rutherford B. Hayes": 0.824,
    "Zachary Taylor": 0.671,
    "William Jefferson Clinton": 0.569,
    "Martin Van Buren": 0.554,
    "Andrew Jackson": 0.516,
    "George W. Bush": 0.391,
    "George Washington": 0.302,
    "Dwight D. Eisenhower": 0.297,
    "John Tyler": 0.283,
    "Chester Arthur": 0.267,
    "Lyndon B. Johnson": 0.173,
    "Gerald Ford": 0.157,
    "Benjamin Harrison": 0.032,
    "James Earl Carter": 0.007,
    "Woodrow Wilson": -0.032,
    "Warren G. Harding": -0.036,
    "Thomas Jefferson": -0.056,
    "Ulysses S. Grant": -0.084,
    "William H. Harrison": -0.158,
    "Abraham Lincoln": -0.321,
    "James Madison": -0.355,
    "Millard Fillmore": -0.388,
    "James K. Polk": -0.388,
    "Richard Nixon": -0.544,
    "Franklin Pierce": -0.553,
    "George H. Bush": -0.619,
    "Grover Cleveland": -0.624,
    "James Monroe": -0.636,
    "James Garfield": -0.664,
    "Harry S. Truman": -0.668,
    "Andrew Johnson": -0.728,
    "Herbert Hoover": -0.866,
    "John Adams": -0.927,
    "James Buchanan": -0.942,
    "William McKinley": -0.996,
    "Calvin Coolidge": -1.175,
    "John Q. Adams": -1.234,
    "William H. Taft": -1.579
}


#Lilienfeld Model

In [33]:
lil_dict = {}
lil_dict["presidents"] = [name for name,score in lilienfeld_scores.items() if name in texts.keys()]
lil_dict["scores"] = [v for k,v in lilienfeld_scores.items() if k in texts.keys()]
lil_dict["texts"] = [texts[k] for k,v in lilienfeld_scores.items() if k in texts.keys()]


##### Enrichment

In [34]:
lil_df = pd.DataFrame(lil_dict)

In [35]:
lil_df["complex"] = lil_df["texts"].apply(complexity)
lil_df["pos"] = lil_df["texts"].apply(pos_tag)

In [36]:
lil_df.head()

presidents  scores  \
0            John F. Kennedy   1.408   
1      Franklin D. Roosevelt   1.079   
2              Ronald Reagan   0.912   
3  William Jefferson Clinton   0.569   
4             George W. Bush   0.391   

                                               texts   complex  \
0  In the 1st place I think the problems are more...  4.271290   
1  Hello Hello Greta I did I called with the fact...  3.958478   
2  Well She core of course was a major part I had...  3.815303   
3  Most people think you know that uh except for ...  3.929799   
4  Well thanks Rudy and thanks George Let me make...  3.877451   

                                                 pos  
0  ADP DET ADJ NOUN PRON VERB DET NOUN AUX ADV AD...  
1  INTJ INTJ PROPN PRON AUX PRON VERB ADP DET NOU...  
2  INTJ PRON NOUN ADP NOUN AUX DET ADJ NOUN PRON ...  
3  ADJ NOUN VERB PRON VERB SCONJ INTJ SCONJ ADP P...  
4  INTJ NOUN PROPN CCONJ NOUN PROPN VERB PRON VER...

In [37]:
# Now integrate the other information sources
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Create a MinMaxScaler instance
scaler = MinMaxScaler()
numeric_pipe = make_pipeline(scaler)

vect1 = TfidfVectorizer(max_features=10000, use_idf=True, ngram_range=(1,2), lowercase=True, tokenizer= str.split)

vect2 = TfidfVectorizer(lowercase=False, tokenizer= str.split)

In [42]:
df = lil_df # Load your data
lil_df

presidents  scores  \
0             John F. Kennedy   1.408   
1       Franklin D. Roosevelt   1.079   
2               Ronald Reagan   0.912   
3   William Jefferson Clinton   0.569   
4              George W. Bush   0.391   
5        Dwight D. Eisenhower   0.297   
6           Lyndon B. Johnson   0.173   
7                 Gerald Ford   0.157   
8           James Earl Carter   0.007   
9               Richard Nixon  -0.544   
10             George H. Bush  -0.619   
11            Harry S. Truman  -0.668   
12             Herbert Hoover  -0.866   

                                                texts   complex  \
0   In the 1st place I think the problems are more...  4.271290   
1   Hello Hello Greta I did I called with the fact...  3.958478   
2   Well She core of course was a major part I had...  3.815303   
3   Most people think you know that uh except for ...  3.929799   
4   Well thanks Rudy and thanks George Let me make...  3.877451   
5   Well I dont know As far as Vice president Nixo...  3.791525   
6   I want a couple of maybe 3 of the light brown ...  3.528333   
7   It was August 1 1974 and uh general Hague who ...  4.219844   
8   Well there are varying degrees of relationship...  4.418886   
9   Because the United States is concerned by more...  3.997506   
10  I dont I honestly Larry I dont I had my chance...  3.636364   
11  Well I I itll sound as if Im Bragging about th...  3.931000   
12  Republic after the First World War After the F...  4.051451   

                                                  pos  
0   ADP DET ADJ NOUN PRON VERB DET NOUN AUX ADV AD...  
1   INTJ INTJ PROPN PRON AUX PRON VERB ADP DET NOU...  
2   INTJ PRON NOUN ADP NOUN AUX DET ADJ NOUN PRON ...  
3   ADJ NOUN VERB PRON VERB SCONJ INTJ SCONJ ADP P...  
4   INTJ NOUN PROPN CCONJ NOUN PROPN VERB PRON VER...  
5   INTJ PRON AUX PART VERB ADV ADV SCONJ PROPN PR...  
6   PRON VERB DET NOUN ADP ADV NUM ADP DET ADJ ADJ...  
7   PRON AUX PROPN NUM NUM CCONJ INTJ ADJ PROPN PR...  
8   INTJ PRON VERB VERB NOUN ADP NOUN ADP PROPN SC...  
9   SCONJ DET PROPN PROPN AUX VERB ADP ADJ ADP ADJ...  
10  PRON VERB PART PRON ADV PROPN PRON VERB PART P...  
11  INTJ PRON PRON PRON AUX VERB SCONJ SCONJ PRON ...  
12  PROPN ADP DET PROPN PROPN PROPN ADP DET PROPN ...

In [47]:
df = lil_df # Load your data

# Assuming 'X' is your DataFrame with a "presidents" column and a "scores" column

# Step 1: Get unique values from the "presidents" column
unique_presidents = lil_df['presidents'].unique()

# Step 2: Randomly select a subset of unique values for the test set
test_presidents = ['Herbert Hoover', 'John F. Kennedy', 'Lyndon B. Johnson']  # Replace with your desired president names

# Step 3: Filter the DataFrame to create X_test based on the selected presidents
X_test = lil_df[lil_df['presidents'].isin(test_presidents)]

# Remove the selected test rows from the original DataFrame to create X_train
X_train = lil_df[~lil_df['presidents'].isin(test_presidents)]

print(X_train)

# Specify the "scores" column as the target variable 'y'
y_train = X_train['scores']
y_test = X_test['scores']









preprocessor = make_column_transformer(
    (numeric_pipe, ['complex']), # we scale the numeric features # 'avg_word_length','avg_sent_length', 'doc_length',
    (vect1, 'texts'), # This is the BoW
    (vect2, 'pos'), # These are the POS patterns
    remainder = 'drop')

# SVR pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

# Parameter grid for RandomizedSearchCV
param_grid = {
    'regressor__C': [0.1, 1, 10, 100],
    'regressor__epsilon': [0.01, 0.1, 1],
    'regressor__kernel': ['linear', 'rbf'],
    'preprocessor__tfidfvectorizer-1__max_features': [1000, 5000, 10000],  # Corrected parameter
}


                   presidents  scores  \
1       Franklin D. Roosevelt   1.079   
2               Ronald Reagan   0.912   
3   William Jefferson Clinton   0.569   
4              George W. Bush   0.391   
5        Dwight D. Eisenhower   0.297   
7                 Gerald Ford   0.157   
8           James Earl Carter   0.007   
9               Richard Nixon  -0.544   
10             George H. Bush  -0.619   
11            Harry S. Truman  -0.668   

                                                texts   complex  \
1   Hello Hello Greta I did I called with the fact...  3.958478   
2   Well She core of course was a major part I had...  3.815303   
3   Most people think you know that uh except for ...  3.929799   
4   Well thanks Rudy and thanks George Let me make...  3.877451   
5   Well I dont know As far as Vice president Nixo...  3.791525   
7   It was August 1 1974 and uh general Hague who ...  4.219844   
8   Well there are varying degrees of relationship...  4.418886   
9   Because 

In [48]:
# RandomizedSearchCV
pipe = RandomizedSearchCV(pipeline, param_grid, cv=10, n_iter=10, random_state=42)

# Fit the model
pipe.fit(X_train, y_train)


best_pipeline = pipe.best_estimator_

# Make predictions on the testing data using the best pipeline
y_pred = best_pipeline.predict(X_test)

print(y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist

[0.2055 0.2055 0.2055]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [49]:
print(y_pred)

[0.2055 0.2055 0.2055]


In [26]:
# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


# Calculate R^2
r2 = r2_score(y_test, y_pred)
print("R^2:", r2)

# Get the best hyperparameters from the random search
best_params = pipe.best_params_
print("Best Hyperparameters:", best_params)

Mean Squared Error: 0.8561780324319648
Mean Absolute Error: 0.9252988881609903
R^2: nan
Best Hyperparameters: {'regressor__kernel': 'linear', 'regressor__epsilon': 1, 'regressor__C': 0.1, 'preprocessor__tfidfvectorizer-1__max_features': 1000}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [ ]:
!pip install openai

In [ ]:
import openai

# Set your API key here
openai.api_key = 'your-api-key'

# Create a query
response = openai.Completion.create(
  engine="gpt-3.5-turbo",  # Specify the GPT-3.5-Turbo model
  prompt="Translate the following English text to French: '{}'",  # Example prompt
  max_tokens=60
)

# Print the response
print(response.choices[0].text.strip())


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-4c4d63f4c4e8>", line 1, in <cell line: 1>
    import openai
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    s